In [ ]:
try:
    import cPickle as pickle
except:
    import pickle
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
from tqdm import tqdm
from shapely.geometry import shape, Polygon, MultiPolygon, Point
from descartes import PolygonPatch
from pandas import DataFrame, Series
import pandas as pd
pd.set_option('io.hdf.default_format', 'table')
import numpy as np
from pyproj import Proj
import rasterio
import rasterio.features
import sys
import os
from datetime import datetime, timedelta
import h5py
import bcolz
import random
sys.path.append('../pyx')
sys.path.append('../py')
import subprocess
os.chdir('../pyx')
subprocess.call(['python', 'setup_cmodels.py', 'build_ext', '-if'])
os.chdir('../ipynb')
import cmodels as cmod
import models as mod
from delineate import Delineate
import calibration as cal
from multiprocessing import Pool
from scipy import optimize
from numba import jit

%matplotlib inline

# Read the basin partition

In [ ]:
try:
    with open('../data/amazon/partition.pkl', 'rb') as f:
        ws = pickle.load(f)
except:
    d = Delineate()
    d.delineate(-1.914, -53.84)
    with open('../data/amazon/partition.pkl', 'wb') as f:
        pickle.dump(d.watersheds, f)

In [ ]:
df_ws = DataFrame()
df_ws['odr'] = [str(i['order']).replace('[', '').replace(']', '').replace(' ', '') for i in ws]
df_ws['out'] = [i['outlet'] for i in ws]
#df_ws['geo'] = [i['geometry'] for i in ws]
df_ws['mask'] = [i['mask'] for i in ws]
df_ws['latlon'] = [i['latlon'] for i in ws]
if not True:
    df_ws['geo'] = None
    for i, j in enumerate(ws):
        this_ws = [j['geometry']]
        done = False
        ws_list = []
        ws_i = 0
        while not done:
            if type(this_ws[ws_i]) == Polygon:
                if this_ws[ws_i].is_valid:
                    ws_list.append(this_ws[ws_i])
                    ws_i += 1
                else:
                    this_ws[ws_i] = this_ws[ws_i].buffer(0)
            else: # expand multi-polygon
                this_ws = this_ws[:ws_i] + [k for k in this_ws[ws_i]] + this_ws[ws_i + 1:]
            if ws_i == len(this_ws):
                done = True
        df_ws.loc[i, 'geo'] = ws_list
df_ws = df_ws.set_index('odr')

# Show the basin partition

In [ ]:
lat0, lat1, lon0, lon1 = -21, 6, -80, -52 # bounding box in degrees
gpm_res = 0.1 # IMERG resolution in degrees
pet_res = 1 / 120 # PET resolution in degrees (0.008333)
acc_res = 1 / 120 # flow accumulation resolution in degrees (0.008333)

In [ ]:
def show_ws(odrs):
    plt.close('all')
    fig = plt.figure()
    ax = fig.add_subplot(111)
    m = Basemap(projection = 'cyl', resolution = 'l', area_thresh = 10000, llcrnrlat = lat0, urcrnrlat = lat1, llcrnrlon = lon0, urcrnrlon = lon1)
    m.drawcoastlines()
    m.drawcountries()

    this_mask = np.zeros((int((lat1 - lat0) / acc_res), int((lon1 - lon0) / acc_res)), dtype = 'uint8')
    odr_max = 0
    for i, this_odr in enumerate(odrs):
        #this_ws = this_row[1]['geo']
        this_out = df_ws.loc[this_odr, 'out']
        mask = df_ws.loc[this_odr, 'mask']
        latlon = df_ws.loc[this_odr,  'latlon']
        if not True:
            if i == 0:
                allWs = this_ws[0]
            for pol in this_ws:
                allWs = allWs.union(pol)
        this_len = len(this_odr)
        if this_len > odr_max:
            odr_max = this_len
        #if type(this_ws) == Polygon:
        #    ws_list = [this_ws]
        #else:
        #    ws_list = this_ws
        y0 = int(round((lat1 - latlon[0]) / acc_res))
        y1 = y0 + mask.shape[0]
        x0 = int(round((latlon[1] - lon0) / acc_res))
        x1 = x0 + mask.shape[1]
        this_mask[y0:y1, x0:x1] = this_mask[y0:y1, x0:x1] + mask * int(np.random.uniform(1, 255))
        if not True:
            for pol in ws_list:
                polygon = PolygonPatch(pol)
                polygon.set_facecolor('white')
                #polygon.set_edgecolor('white')
                #polygon.set_linewidth(0.01)
                ax.add_patch(polygon)
        #ax.scatter(this_out[1], this_out[0], color = 'black', alpha = .3, zorder = 10)

    this_mask = np.where(this_mask == 0, np.nan, this_mask)
    m.imshow(this_mask, origin = 'upper', interpolation = 'nearest', zorder = 10, alpha = 1)

    plt.show()

# Compute every subbasin's mask
It is a np.array in the native IMERG resolution, so that masking the precipitation over the subbasin is just a multiplication. The boundary pixels are a fraction of the precipitation pixels, according to the area covered by the subbasin inside each pixel.

Keep in mind that this doesn't work (yet) for subbasins which have a hole.

In [ ]:
try:
    df_ws = pd.read_pickle('../data/amazon/amazon.pkl')
except:
    df_ws['gpm_mask'] = None
    df_ws['pet_mask'] = None
    df_ws['acc_mask'] = None
    df_ws['gpm_view'] = None
    df_ws['pet_view'] = None
    with rasterio.drivers():
        with rasterio.open('../data/amazon/sa_acc_30s/sa_acc_30s/w001001.adf') as src:
            data, = src.read()
            acc_aff = src.affine
    
    x0, y0 = [int(round(i)) for i in ~acc_aff * (lon0, lat1)]
    x1, y1 = [int(round(i)) for i in ~acc_aff * (lon1, lat0)]
    acc = data[y0:y1, x0:x1]
    for ws_i in tqdm(range(len(df_ws))):
        this_odr = df_ws.index[ws_i]
        this_mask = df_ws.iloc[ws_i]['mask']

        # subbasin's bounding box:
        lt1, ln0 = df_ws.iloc[ws_i]['latlon']
        lt0 = lt1 - acc_res * this_mask.shape[0]
        ln1 = ln0 + acc_res * this_mask.shape[1]
            
        gpm_mask = np.zeros((int(round((lat1 - lat0) / gpm_res)), int(round((lon1 - lon0) / gpm_res))))
        pet_mask = np.zeros((int(round((lat1 - lat0) / pet_res)), int(round((lon1 - lon0) / pet_res))))
        acc_mask = np.zeros_like(this_mask, dtype = 'int32')
        for res in set([gpm_res, pet_res]):
            x0 = int(np.floor((ln0 - lon0) / res))
            x1 = int(np.ceil((ln1 - lon0) / res))
            y0 = int(np.floor((lat1 - lt1) / res))
            y1 = int(np.ceil((lat1 - lt0) / res))
            if res == gpm_res:
                df_ws.loc[this_odr, 'gpm_view'] = [x0, x1, y0, y1]
            if res == pet_res:
                df_ws.loc[this_odr, 'pet_view'] = [x0, x1, y0, y1]
            for y in range(y0, y1):
                for x in range(x0, x1):
                    yy0 = int(round((lt1 - (lat1 - y * res)) / acc_res))
                    yy1 = int(round((lt1 - (lat1 - (y + 1) * res)) / acc_res))
                    xx0 = int(round(((lon0 + x * res) - ln0) / acc_res))
                    xx1 = int(round(((lon0 + (x + 1) * res) - ln0) / acc_res))
                    xxyy = [xx0, xx1, yy0, yy1]
                    xxyy[0] = np.max([0, xxyy[0]])
                    xxyy[1] = np.min([this_mask.shape[1], xxyy[1]])
                    xxyy[2] = np.max([0, xxyy[2]])
                    xxyy[3] = np.min([this_mask.shape[0], xxyy[3]])
                    xx0, xx1, yy0, yy1 = xxyy
                    res_ratio = int(round(res / acc_res))
                    if res == gpm_res:
                        gpm_mask[y, x] = float(np.sum(this_mask[yy0:yy1, xx0:xx1])) / (res_ratio * res_ratio)
                    if res == pet_res:
                        pet_mask[y, x] = float(np.sum(this_mask[yy0:yy1, xx0:xx1])) / (res_ratio * res_ratio)

        x0, x1, y0, y1 = df_ws.loc[this_odr, 'gpm_view']
        df_ws.loc[this_odr, 'gpm_mask'] = gpm_mask[y0:y1, x0:x1].copy()
        x0, x1, y0, y1 = df_ws.loc[this_odr, 'pet_view']
        df_ws.loc[this_odr, 'pet_mask'] = pet_mask[y0:y1, x0:x1].copy()
        x0 = int(round((ln0 - lon0) / acc_res))
        x1 = int(round((ln1 - lon0) / acc_res))
        y0 = int(round((lat1 - lt1) / acc_res))
        y1 = int(round((lat1 - lt0) / acc_res))
        df_ws.loc[this_odr, 'acc_view'] = [x0, x1, y0, y1]
        df_ws.loc[this_odr, 'acc_mask'] = acc[y0:y1, x0:x1] * this_mask

        if not True:
            this_ws = df_ws.iloc[ws_i]['geo']
            plt.close('all')
            fig = plt.figure()
            ax = fig.add_subplot(111)
            m = Basemap(projection = 'cyl', resolution = 'l', area_thresh = 10000, llcrnrlat = lat0, urcrnrlat = lat1, llcrnrlon = lon0, urcrnrlon = lon1)
            m.drawcoastlines()
            m.drawcountries()
            if type(this_ws) == Polygon:
                ws_list = [this_ws]
            else:
                ws_list = this_ws
            for pol in ws_list:
                polygon = PolygonPatch(pol, zorder = 1)
                polygon.set_facecolor('white')
                ax.add_patch(polygon)
            m.imshow(np.where(gpm_mask == 0, np.nan, gpm_mask), cmap = 'binary', interpolation = 'nearest', origin = 'upper', alpha = 0.5, zorder = 10)
            plt.show()

    df_ws.to_pickle('../data/amazon/amazon.pkl')

Check if the sum of all subbasin masks is the whole basin. It should be 1 inside (of course not at the edge).

In [ ]:
plt.close('all')
mask = np.zeros((int((lat1 - lat0) / gpm_res), int((lon1 - lon0) / gpm_res)))
for this_row in df_ws.iterrows():
    this_mask = this_row[1]['gpm_mask']
    x0, x1, y0, y1 = this_row[1]['gpm_view']
    mask[y0:y1, x0:x1] = mask[y0:y1, x0:x1] + this_mask
plt.imshow(mask, interpolation = 'nearest')
plt.show()

# Get the precipitation over each subbasin
We take the subset of the precipitation array that corresponds to the subbasin array, and just multiply them and take the mean.

In [ ]:
try:
    df_imerg = pd.read_pickle('../data/amazon/imerg.pkl')
except:
    rootDir = '/media/disk/data/imerg/late/'
    if os.path.exists(rootDir):
        dates = []
        files = []
        for dirName, subdirList, fileList in os.walk(rootDir):
            for fname in fileList:
                if fname.endswith('RT-H5'):
                    year = int(fname[23:27])
                    month = int(fname[27:29])
                    day = int(fname[29:31])
                    hour = int(fname[33:35])
                    minu = int(fname[35:37])
                    dates.append(datetime(year, month, day, hour, minu))
                    files.append(dirName + '/' + fname)
        df_imerg = DataFrame(data = files, index = dates, columns = ['file']).sort_index()
        # check whether there are missing/duplicate dates:
        date_range = pd.date_range(df_imerg.index[0], df_imerg.index[-1], freq = '30min')
        for this_date in date_range:
            if this_date not in df_imerg.index:
                print(str(this_date) + ' not found')
            elif len(df_imerg.loc[this_date]) > 1:
                print(str(this_date) + ' appears multiple times')
    df_imerg.to_pickle('../data/amazon/imerg.pkl')

Now get the mean precipitation over each subbasin.

In [ ]:
try:
    ca_gpm = bcolz.open(rootdir = '../data/amazon/ca_gpm', mode = 'r')
except:
    res = gpm_res
    x0 = int((lon0 - (-180)) / res)
    x1 = int((lon1 - (-180)) / res)
    y0 = int((90 - lat1) / res)
    y1 = int((90 - lat0) / res)

    # in order to minimize disk access, we get the precipitations of all subbasins
    # for a precipitation file (i.e. a date), then the next date, etc.
    ca_gpm = bcolz.zeros((0, len(df_ws)), dtype = 'f8', rootdir = '../tmp/ca_gpm', mode = 'w')
    for this_date in tqdm(df_imerg.index):
        data_loaded = False
        df = np.empty(len(df_ws.index))
        for odr_i, this_odr in enumerate(df_ws.index):
            this_mask = df_ws.loc[this_odr, 'gpm_mask']
            if not data_loaded:
                f = h5py.File(df_imerg.loc[this_date, 'file'], 'r')
                data = np.array(f['Grid/precipitationCal']).transpose()[::-1][y0:y1, x0:x1]
                data = np.clip(data, 0, np.inf)
                f.close()
                data_loaded = True
                np.save('../data/amazon/shrink/p_' + str(this_date).replace(' ', '_'), data)
            xx0, xx1, yy0, yy1 = df_ws.loc[this_odr, 'gpm_view']
            p = np.sum(data[yy0:yy1, xx0:xx1] * this_mask) / np.sum(this_mask)
            df[odr_i] = p
        ca_gpm.append(df)
    ca_gpm.flush()
    # transpose carray:
    ca_gpm2 = bcolz.zeros((0, ca_gpm.shape[0]), dtype = 'f4', rootdir = '../data/amazon/ca_gpm', mode = 'w')
    for i in tqdm(range(ca_gpm.shape[1])):
        ca_gpm2.append(ca_gpm[:, i])
    ca_gpm2.flush()

In [ ]:
def get_odr_startswith(odrs, odr):
    odr_list = []
    for this_odr in odrs:
        if this_odr.startswith(odr):
            odr_list.append(this_odr)
    return odr_list

def substract_odr(odr1, odr2):
    odr = []
    for this_odr in odr1:
        if this_odr not in odr2:
            odr.append(this_odr)
    return odr

def intersect_odr(odr1, odr2):
    odr = []
    for this_odr in odr1:
        if this_odr in odr2:
            odr.append(this_odr)
    return odr

def get_p_imerg(df_ws, df_imerg, ca_gpm, odr):
    first_time = True
    pix_sum = 0.
    for odr_i, this_odr in enumerate(df_ws.index):
        if this_odr in odr:
            this_mask = df_ws.loc[this_odr, 'mask']
            this_p = ca_gpm[odr_i] * np.sum(this_mask)
            pix_sum += np.sum(this_mask)
            if first_time:
                first_time = False
                p_imerg = this_p
            else:
                p_imerg = p_imerg + this_p
    p_imerg = Series(data = p_imerg / pix_sum, index = df_imerg.index)
    if p_imerg.index[0].to_datetime().minute != 0:
        p_imerg = p_imerg.iloc[1:]
    p_imerg = p_imerg.reindex(pd.date_range(p_imerg.index[0], p_imerg.index[-1], freq = '30min'))
    p_imerg = p_imerg.interpolate()
    p_imerg = pd.rolling_mean(p_imerg, window = 3, min_periods = 3, center = True)
    p_imerg = p_imerg.iloc[1::2].dropna()
    p_imerg.index = (p_imerg.index - timedelta(minutes = 30)).tz_localize('UTC')
    return p_imerg

def get_pet(df_ws, df_imerg, df_pet, odr):
    pix_sum = 0.
    months = np.array([this_date.month for this_date in df_imerg.index], dtype = 'uint8')
    this_pet = np.empty(len(df_imerg.index))
    pet = np.zeros_like(this_pet)
    for this_odr in df_ws.index:
        if this_odr in odr:
            this_mask = df_ws.loc[this_odr, 'mask']
            for i in range(1, 12 + 1):
                this_pet[np.where(months == i)] = df_pet.loc[i, this_odr]
            this_pet = this_pet * np.sum(this_mask)
            pix_sum += np.sum(this_mask)
            pet = pet + this_pet
    pet = Series(data = pet / pix_sum, index = df_imerg.index)
    if pet.index[0].to_datetime().minute != 0:
        pet = pet.iloc[1:]
    pet = pet.reindex(pd.date_range(pet.index[0], pet.index[-1], freq = '30min'))
    pet = pet.interpolate()
    pet = pd.rolling_mean(pet, window = 3, min_periods = 3, center = True)
    pet = pet.iloc[1::2].dropna()
    pet.index = (pet.index - timedelta(minutes = 30)).tz_localize('UTC')
    return pet

def get_area(df_ws, odr):
    area = 0.
    for this_odr in odr:
        area += np.sum(df_ws.loc[this_odr, 'mask'])
    return area

In [ ]:
try:
    df_pet = pd.read_pickle('../data/amazon/pet.pkl')
except:
    res = pet_res
    x0 = int((lon0 - (-180)) / res)
    x1 = int((lon1 - (-180)) / res)
    y0 = int((90 - lat1) / res)
    y1 = int((90 - lat0) / res)

    df_pet = DataFrame(columns = df_ws.index, index = range(1, 12 + 1))
    for this_month in tqdm(range(1, 12 + 1)):
        try:
            del data
        except:
            pass
        with rasterio.drivers():
            with rasterio.open('../data/amazon/PET_he_monthly/pet_he_' + str(this_month) + '/w001001.adf') as src:
                data, = src.read()
        yy0, yy1 = int(round((lat1 - src.affine[5]) / src.affine[4])), int(round((lat0 - src.affine[5]) / src.affine[4]))
        xx0, xx1 = int(round((lon0 - src.affine[2]) / src.affine[0])), int(round((lon1 - src.affine[2]) / src.affine[0]))
        data = data[y0:y1, x0:x1]
        np.clip(data, 0, np.inf, out = data)
        for this_odr in df_ws.index:
            this_mask = df_ws.loc[this_odr, 'pet_mask']
            xx0, xx1, yy0, yy1 = df_ws.loc[this_odr, 'pet_view']
            pet = np.sum(data[yy0:yy1, xx0:xx1] * this_mask) / np.sum(this_mask) / 30 / 24 # in mm/h
            df_pet.loc[this_month, this_odr] = pet
    df_pet.to_pickle('../data/amazon/pet.pkl')

In [ ]:
try:
    peq = pd.read_pickle('../data/amazon/peq.pkl')
except:
    df = pd.read_csv('../data/amazon/R_amz_amz_ja2_0215_01.txt', skiprows = 20, sep = ';', header = None, names = ['col0', 'date', 'time', 'height', 'col4', 'col5', 'col6'])
    df = df[['date', 'time', 'height']]
    dates = []
    for i in df.index:
        this_date = df.loc[i, 'date'].replace(' ', '')
        this_time = df.loc[i, 'time'].replace(' ', '')
        dates.append(datetime(year = int(this_date[:4]), month = int(this_date[5:7]), day = int(this_date[8:]), hour = int(this_time[:2])))
    df.index = dates
    df = df['height']
    df = df.reindex(pd.date_range(df.index[0], df.index[-1], freq = '1H'))
    df.index = df.index.tz_localize('UTC')
    #df_q = df.interpolate()
    df_q = df

    p_imerg = get_p_imerg(df_ws, df_imerg, ca_gpm, get_odr_startswith(df_ws.index, '0'))
    pet = get_pet(df_ws, df_imerg, df_pet, get_odr_startswith(df_ws.index, '0'))
    peq = DataFrame()
    peq['p'] = p_imerg
    peq['e'] = pet
    peq['q'] = df_q

    peq.to_pickle('../data/amazon/peq.pkl')

In [ ]:
def set_cal(x_ranges):
    df_cal = DataFrame()
    x_sample_nb = 100
    for x_i in range(4):
        param = 'x' + str(x_i + 1)
        df_cal[param + '_range'] = np.linspace(x_ranges[x_i][0], x_ranges[x_i][1], x_sample_nb)
        df_cal[param + '_score'] = -np.inf
    return df_cal

@jit
def dist_map(dst, src):
    ret = np.empty_like(dst)
    dist0 = np.empty_like(src)
    dist1 = np.empty_like(dst)
    nb = 0
    for i in range(len(src)):
        if (not np.isnan(src[i])) and (not np.isnan(dst[i])):
            dist0[nb] = src[i]
            dist1[nb] = dst[i]
            nb += 1
    dist0[:nb].sort()
    dist1[:nb].sort()
    for i in range(len(dst)):
        if np.isnan(dst[i]):
            ret[i] = np.nan
        else:
            j = 0
            done = False
            while not done:
                if dst[i] == dist1[j]:
                    done = True
                else:
                    j += 1
            ret[i] = dist0[j]
    return ret

def single_cal(peq, df_cal):
    x_sample_nb = len(df_cal)
    x = np.empty(4)
    iter_nb = x_sample_nb * 20
    for i in range(iter_nb):
        for x_i in range(4):
            this_range = 'x' + str(x_i + 1) + '_range'
            x[x_i] = random.uniform(df_cal.loc[0, this_range], df_cal.loc[x_sample_nb - 1, this_range])
        q_mod = mod.gr4h(x)
        # warm-up:
        q_mod.run(peq)
        q_mod.run(peq)
        q_mod.run(peq)
        q_sim = q_mod.run(peq)
        q_obs = dist_map(peq['q'].values, q_sim.values)

        warmup = 24 * 30
        score = cal.nse(q_obs[warmup:], q_sim.values[warmup:])
        for x_i in range(4):
            done = False
            k = 1
            while not done:
                if x[x_i] <= df_cal.loc[k, 'x' + str(x_i + 1) + '_range']:
                    done = True
                else:
                    k += 1
            this_score = 'x' + str(x_i + 1) + '_score'
            if score > df_cal.loc[k, this_score]:
                df_cal.loc[k, this_score] = score
    return df_cal

def dual_cal(peq, df_cal, df_x, proc_nb):
    x_sample_nb = len(df_cal)
    iter_nb = int(x_sample_nb * 30 / proc_nb)
    p = [None, None]
    e = [None, None]
    p[0] = peq.loc[:, 'p0'].values
    p[1] = peq.loc[:, 'p1'].values
    e[0] = peq.loc[:, 'e0'].values
    e[1] = peq.loc[:, 'e1'].values
    q_cal = peq.loc[:, 'q_cal'].values
    q = peq.loc[:, 'q'].values
    len_peq = len(peq)
    q_sim = [np.empty(len_peq * 4), np.empty(len_peq * 4)]
    ind = np.array([[0 ,0 ,0 ,0 ,0], [0, 0, 0, 0, 0]], dtype = 'int')
    for i in tqdm(range(iter_nb)):
        for ws_i in range(2):
            for x_i in range(5):
                param = 'x' + str(ws_i) + '_' + str(x_i + 1)
                this_range = param + '_range'
                delta = abs(df_cal.loc[0, this_range] - df_cal.loc[1, this_range]) / 2
                this_rand = random.randint(0, x_sample_nb - 2)
                df_x.loc[ws_i, 'x' + str(x_i + 1)] = df_cal.loc[this_rand, this_range] + delta
                ind[ws_i][x_i]= this_rand
            df_x.loc[0, 'x5'] = 0.
            this_x = [df_x.loc[ws_i, 'x' + str(x_i + 1)] for x_i in range(4)]
            #this_peq = peq.loc[:, ['p' + str(ws_i), 'e' + str(ws_i)]].rename(columns = {'p' + str(ws_i): 'p', 'e' + str(ws_i): 'e'})
            q_mod = cmod.gr4h(this_x)
            q_sim[ws_i][0 * len_peq:1 * len_peq] = q_mod.run([p[ws_i], e[ws_i]])
            q_sim[ws_i][1 * len_peq:2 * len_peq] = q_mod.run([p[ws_i], e[ws_i]])
            q_sim[ws_i][2 * len_peq:3 * len_peq] = q_mod.run([p[ws_i], e[ws_i]])
            q_sim[ws_i][3 * len_peq:4 * len_peq] = q_mod.run([p[ws_i], e[ws_i]])
            q_sim[ws_i] = cmod.delay([df_x.loc[ws_i, 'x5'] + df_x.loc[ws_i, 'x5+']]).run([q_sim[ws_i] * df_x.loc[ws_i, 'weight']])
        q_sim2 = q_sim[0][-len_peq:] + q_sim[1][-len_peq:] + q_cal
        q_obs = dist_map(q, q_sim2)

        ind[0][4] = 0
        warmup = 24 * 30
        score = cal.nse(q_obs[warmup:], q_sim2[warmup:])
        for ws_i in range(2):
            for x_i in range(5):
                param = 'x' + str(ws_i) + '_' + str(x_i + 1)
                if score > df_cal.loc[ind[ws_i][x_i], param + '_score']:
                    df_cal.loc[ind[ws_i][x_i], param + '_score'] = score
    return df_cal

def get_mse(x, peq):
    x_range = [
        [0., np.inf],
        [-np.inf, np.inf],
        [0., np.inf],
        [0., np.inf],
        [0., np.inf],
        [-np.inf, np.inf],
        [0., np.inf]
        ]
    # forbidden x values:
    for i in range(7):
        if x[i] < x_range[i][0]:
            return np.inf
        if x[i] > x_range[i][1]:
            return np.inf
    q_mod = mod.gr4h(x[:4])
    # warm-up:
    q_mod.run(peq)
    q_mod.run(peq)
    q_mod.run(peq)
    q_sim = q_mod.run(peq)
    q_obs = dist_map(peq['q'].values, q_sim.values)

    warmup = 24 * 30
    score = cal.mse(q_obs[warmup:], q_sim.values[warmup:])
    return score

In [ ]:
try:
    df_cal = pd.read_pickle('../data/amazon/df_cal.pkl')
except:
    x_ranges = [[300, 700], [-1, 1], [1200, 2200], [150, 250]]
    df_cal = set_cal(x_ranges)
    while True:
        proc_nb = 3
        pool = Pool(processes = proc_nb)
        result = pool.starmap(single_cal, [(peq, df_cal.copy()) for proc in range(proc_nb)])
        for x_i in range(4):
            this_score = 'x' + str(x_i + 1) + '_score'
            for this_df in result:
                df_cal.loc[:, this_score] = np.where(this_df.loc[:, this_score] > df_cal.loc[:, this_score], this_df.loc[:, this_score], df_cal.loc[:, this_score])
        pool.close()
        pool.join()
        # plot parameters:
        x_sample_nb = len(df_cal)
        plt.close('all')
        plt.figure()
        for i in range(4):
            a = 4 * 100 + 11 + i
            plt.subplot(a)
            delta = abs(df_cal.loc[0, 'x' + str(i + 1) + '_range'] - df_cal.loc[1, 'x' + str(i + 1) + '_range']) / 2
            plt.plot(df_cal.loc[:x_sample_nb - 2, 'x' + str(i + 1) + '_range'] + delta, df_cal.loc[1:, 'x' + str(i + 1) + '_score'])
            if not True:
                for this_df in result:
                    plt.plot(this_df.loc[:x_sample_nb - 2, 'x' + str(i + 1) + '_range'] + delta, this_df.loc[1:, 'x' + str(i + 1) + '_score'])
            plt.title('X' + str(i + 1))
        plt.show()
        # get best parameters:
        x = [0] * 4
        for x_i in range(4):
            this_range = 'x' + str(x_i + 1) + '_range'
            this_score = 'x' + str(x_i + 1) + '_score'
            delta = abs(df_cal.loc[0, this_range] - df_cal.loc[1, this_range]) / 2
            x[x_i] = df_cal.loc[np.argmax(df_cal.loc[:, this_score]), this_range] - delta
        # plot simulation with best parameters:
        q_mod = mod.gr4h(x[:4])
        # warm-up:
        q_mod.run(peq)
        q_mod.run(peq)
        q_mod.run(peq)
        q_sim = q_mod.run(peq)
        q_obs = dist_map(peq.loc[:, 'q'].values, q_sim.values)
        warmup = 24 * 30
        score = cal.nse(q_obs[warmup:], q_sim.values[warmup:])
        print('X = ' + str(x))
        print('NSE = ' + str(score))
        peq['q_sim'] = q_sim
        peq['q_obs'] = q_obs
        #peq[['p', 'e', 'q_sim']].plot()
        peq[['q_sim']].plot(color = 'red')
        plt.scatter(peq.index, peq['q_obs'].values, color = 'blue')
        plt.title('NSE = ' + str(score))
        plt.show()
        # adjust parameters:
        x_ranges = [
            [max(0, x[0] - 100), x[0] + 100],
            [x[1] - 0.1, x[1] + 0.1],
            [max(0, x[2] - 10), x[2] + 10],
            [max(0, x[3] - 1), x[3] + 1]
            ]
        df_cal = set_cal(x_ranges)
        df_cal.to_pickle('../data/amazon/df_cal.pkl')

In [ ]:
try:
    df_x = pd.read_pickle('../data/amazon/df_x.pkl')
    peq['q_cal'] = pd.read_pickle('../data/amazon/q_cal.pkl')
except:
    # we start from the whole basin
    df_x = DataFrame()
    df_x.loc[0, 'weight'] = 1.
    x0 = [354.54545454545456, -0.28282828282828282, 1780.8080808080806, 191.91919191919192, 0]
    for x_i in range(5):
        df_x.loc[0, 'x' + str(x_i + 1)] = x0[x_i]
    head0 = '0'
    df_x.loc[0, 'head'] = head0
    odr0 = ''
    for this_odr in get_odr_startswith(df_ws.index, head0):
        odr0 += this_odr + ';'
    odr0 = odr0[:-1]
    df_x.loc[0, 'odr'] = odr0
    df_x.loc[0, 'area'] = get_area(df_ws, get_odr_startswith(df_ws.index, head0))
    df_x.loc[0, 'parent'] = -1
    df_x.loc[0, 'x5+'] = 0.
    this_peq = DataFrame()
    this_peq['e'] = get_pet(df_ws, df_imerg, df_pet, get_odr_startswith(df_ws.index, head0))
    this_peq['p'] = get_p_imerg(df_ws, df_imerg, ca_gpm, get_odr_startswith(df_ws.index, head0))
    q_mod = mod.gr4h(x0[:4])
    # warm-up
    q_mod.run(peq)
    q_mod.run(peq)
    q_mod.run(peq)
    q_sim = q_mod.run(this_peq)
    q_obs = dist_map(peq['q'].values, q_sim.values)
    warmup = 24 * 30
    score = cal.nse(q_obs[warmup:], q_sim.values[warmup:])
    df_x.loc[0, 'nse'] = score
    df_x.loc[0, 'split'] = False
    df_x.loc[0, 'valid'] = True
    peq['q_cal'] = q_sim
    peq['q_cal'].to_pickle('../data/amazon/q_cal.pkl')

In [ ]:
#q_obs_keep = []
all_done = False
while not all_done:
    # find the next basin to split
    done = False
    rows = df_x.iterrows()
    ws_i0 = 0
    while not done:
        this_row = next(rows)
        if not this_row[1]['split']:
            done = True
        else:
            ws_i0 += 1
        if ws_i0 == len(df_x):
            done = True
            all_done = True
    if not all_done:
        odr0 = this_row[1]['odr'].split(';')
        if len(odr0) == 1:
            # it only consists of one subbasin, cannot split
            df_x.loc[ws_i0, 'split'] = True
            df_x.loc[ws_i0, 'valid'] = True
        else:
            # split and get rainfall for each half
            x0 = [this_row[1]['x' + str(x_i + 1)] for x_i in range(5)]
            area0 = this_row[1]['area']
            weight0 = this_row[1]['weight']
            head0 = this_row[1]['head']
            area = [None, None]
            odr = [None, None]
            head = [head0, None]
            diff_area = np.inf
            for this_odr in tqdm(odr0):
                odr_1 = intersect_odr(get_odr_startswith(df_ws.index, this_odr), odr0)
                this_area = get_area(df_ws, odr_1)
                diff = abs(area0 / 2 - this_area)
                if diff < diff_area:
                    diff_area = diff
                    head[1] = this_odr
                    area[1] = this_area
                    odr[1] = odr_1
            area[0] = area0 - area[1]
            odr[0] = substract_odr(odr0, odr[1])
            show_ws(odr[0])
            show_ws(odr[1])
            peq['e0'] = get_pet(df_ws, df_imerg, df_pet, odr[0])
            peq['e1'] = get_pet(df_ws, df_imerg, df_pet, odr[1])
            peq['p0'] = get_p_imerg(df_ws, df_imerg, ca_gpm, odr[0])
            peq['p1'] = get_p_imerg(df_ws, df_imerg, ca_gpm, odr[1])
            # remove the part of the catchment we are going to split:
            this_x = [df_x.loc[ws_i0, 'x' + str(x_i + 1)] for x_i in range(4)]
            this_peq = DataFrame()
            this_peq['e'] = get_pet(df_ws, df_imerg, df_pet, df_x.loc[ws_i0, 'odr'].split(';'))
            this_peq['p'] = get_p_imerg(df_ws, df_imerg, ca_gpm, df_x.loc[ws_i0, 'odr'].split(';'))
            q_mod = cmod.gr4h(this_x)
            q_sim = q_mod.run([this_peq.p.values, this_peq.e.values])
            q_sim = np.append(q_sim, q_mod.run([this_peq.p.values, this_peq.e.values]))
            q_sim = np.append(q_sim, q_mod.run([this_peq.p.values, this_peq.e.values]))
            q_sim = np.append(q_sim, q_mod.run([this_peq.p.values, this_peq.e.values]))
            q_sim = q_sim * df_x.loc[ws_i0, 'weight']
            q_sim = cmod.delay([df_x.loc[ws_i0, 'x5+']]).run([q_sim])[-len(this_peq):]
            q_sim = Series(data = q_sim, index = this_peq.index)
            peq['q_cal'] = peq['q_cal'] - q_sim
            # prepare calibration of each half
            x_sample_nb = 100
            xlen = len(df_x)
            retry = True
            while retry:
                df_cal = DataFrame()
                for ws_i in range(2):
                    for x_i in range(5):
                        df_x.loc[xlen + ws_i, 'x' + str(x_i + 1)] = x0[x_i]
                    df_x.loc[xlen + ws_i, 'weight'] = area[ws_i] / area0 * weight0
                    df_x.loc[xlen + ws_i, 'area'] = area[ws_i]
                    df_x.loc[xlen + ws_i, 'head'] = head[ws_i]
                    df_x.loc[xlen + ws_i, 'parent'] = ws_i0
                    odr_str = ''
                    for this_odr in odr[ws_i]:
                        odr_str += this_odr + ';'
                    odr_str = odr_str[:-1]
                    df_x.loc[xlen + ws_i, 'odr'] = odr_str
                    df_x.loc[xlen + ws_i, 'x5'] = 0
                    # before calibration, x5+ doesn't include x5
                    df_x.loc[xlen + ws_i, 'x5+'] = df_x.loc[ws_i0, 'x5+']
                    df_x.loc[xlen + ws_i, 'split'] = False
                    df_x.loc[xlen + ws_i, 'valid'] = False
                    this_x = [df_x.loc[xlen + ws_i, 'x' + str(x_i + 1)] for x_i in range(5)]
                    for x_i in range(5):
                        if x_i == 0:
                            a = max(0, this_x[x_i] - 1000)
                            b = this_x[x_i] + 1000
                        elif x_i == 1:
                            a = this_x[x_i] - 0.5
                            b = this_x[x_i] + 0.5
                        elif x_i == 2:
                            a = max(0, this_x[x_i] - 1000)
                            b = this_x[x_i] + 1000
                        elif x_i == 3:
                            a = max(0, this_x[x_i] - 100)
                            b = this_x[x_i] + 100
                        elif x_i == 4:
                            x5_max = np.inf
                            for i, this_i in enumerate(df_x.index):
                                if df_x.loc[this_i, 'head'].startswith(df_x.loc[xlen + ws_i, 'head']) and df_x.loc[this_i, 'valid'] and i != xlen + ws_i:
                                    if df_x.loc[this_i, 'x5+'] < x5_max:
                                        x5_max = df_x.loc[this_i, 'x5+']
                            if np.isinf(x5_max):
                                b = 1000
                            else:
                                b = x5_max - df_x.loc[ws_i0, 'x5+']
                            a = 0
                        param = 'x' + str(ws_i) + '_' + str(x_i + 1)
                        df_cal[param + '_range'] = np.linspace(a, b, x_sample_nb)
                        df_cal[param + '_score'] = -np.inf
                if (df_x.loc[xlen, 'head'].startswith(df_x.loc[xlen + 1, 'head']) or df_x.loc[xlen + 1, 'head'].startswith(df_x.loc[xlen, 'head'])) and not np.isinf(x5_max):
                    df_cal['x0_5_range'] = np.linspace(0, (x5_max - df_x.loc[ws_i0, 'x5+']) / 2, x_sample_nb)
                    df_cal['x1_5_range'] = np.linspace(0, (x5_max - df_x.loc[ws_i0, 'x5+']) / 2, x_sample_nb)
                # perform calibration
                proc_nb = 3
                if proc_nb == 1:
                    df_cal = dual_cal(peq, df_cal, df_x.loc[xlen:].reset_index(drop = True), proc_nb)
                else:
                    pool = Pool(processes = proc_nb)
                    result = pool.starmap(dual_cal, [(peq, df_cal.copy(), df_x.loc[xlen:].reset_index(drop = True).copy(), proc_nb) for proc in range(proc_nb)])
                    for ws_i in range(2):
                        for x_i in range(5):
                            this_score = 'x' + str(ws_i) + '_' + str(x_i + 1) + '_score'
                            for this_df in result:
                                df_cal.loc[:, this_score] = np.where(this_df.loc[:, this_score] > df_cal.loc[:, this_score], this_df.loc[:, this_score], df_cal.loc[:, this_score])
                    pool.close()
                    pool.join()
                for ws_i in range(2):
                    for x_i in range(5):
                        param = 'x' + str(ws_i) + '_' + str(x_i + 1)
                        max_i = np.argmax(df_cal.loc[:, param + '_score'])
                        delta = abs(df_cal.loc[0, param + '_range'] - df_cal.loc[1, param + '_range']) / 2
                        df_x.loc[xlen + ws_i, 'x' + str(x_i + 1)] = df_cal.loc[max_i, param + '_range'] + delta
                df_x.loc[xlen, 'x5'] = 0.
                for ws_i in range(2):
                    df_x.loc[xlen + ws_i, 'valid'] = True
                    # after calibration, x5+ includes x5
                    df_x.loc[xlen + ws_i, 'x5+'] = df_x.loc[xlen + ws_i, 'x5'] + df_x.loc[ws_i0, 'x5+']
                fig = plt.figure()
                ax = fig.add_subplot(111)
                for ws_i in df_x.index[xlen:]:
                    this_x = [df_x.loc[ws_i, 'x' + str(x_i + 1)] for x_i in range(4)]
                    this_peq = DataFrame()
                    this_peq['e'] = get_pet(df_ws, df_imerg, df_pet, df_x.loc[ws_i, 'odr'].split(';'))
                    this_peq['p'] = get_p_imerg(df_ws, df_imerg, ca_gpm, df_x.loc[ws_i, 'odr'].split(';'))
                    q_mod = cmod.gr4h(this_x)
                    q_sim = q_mod.run([this_peq.p.values, this_peq.e.values])
                    q_sim = np.append(q_sim, q_mod.run([this_peq.p.values, this_peq.e.values]))
                    q_sim = np.append(q_sim, q_mod.run([this_peq.p.values, this_peq.e.values]))
                    q_sim = np.append(q_sim, q_mod.run([this_peq.p.values, this_peq.e.values]))
                    q_sim = cmod.delay([df_x.loc[ws_i, 'x5+']]).run([q_sim])[-len(this_peq):]
                    q_sim = Series(data = q_sim, index = this_peq.index)
                    peq['q' + str(ws_i - xlen)] = q_sim
                    q_sim = q_sim * df_x.loc[ws_i, 'weight']
                    peq['q_cal'] = peq['q_cal'] + q_sim
                q_obs = dist_map(peq['q'].values, peq['q_cal'].values)
                #q_obs_keep.append(Series(q_obs, index = peq.index).interpolate())
                peq['q_obs'] = q_obs
                warmup = 24 * 30
                score = cal.nse(peq['q_obs'].values[warmup:], peq['q_cal'].values[warmup:])
                for ws_i in range(2):
                    df_x.loc[xlen + ws_i, 'nse'] = score
                if True:#score > df_x.loc[ws_i0, 'nse'] - 0.01:
                    retry = False
                else:
                    plt.close('all')
                    print(df_x.loc[xlen:])
            df_x.loc[ws_i0, 'split'] = True
            df_x.loc[ws_i0, 'valid'] = False
            
            peq['q_cal'].plot(ax = ax, color = 'green')
            ax.scatter(peq['q_obs'].index, peq['q_obs'].values, color = 'blue')
            ax.set_title('NSE = ' + str(score))
            plt.show()
            
            #plt.close('all')
            #for this_q in q_obs_keep:
            #    this_q.plot(color = 'grey')
            #plt.show()
            
            if True:
                for ws_i in range(2):
                    plt.close('all')
                    plt.figure()
                    for x_i in range(5):
                        a = 5 * 100 + 11 + x_i
                        param = 'x' + str(ws_i) + '_' + str(x_i + 1)
                        plt.subplot(a)
                        delta = abs(df_cal.loc[0, param + '_range'] - df_cal.loc[1, param + '_range']) / 2
                        plt.plot(df_cal.loc[:x_sample_nb - 2, param + '_range'] + delta, df_cal.loc[1:, param + '_score'])
                        if not True:
                            for this_df in result:
                                plt.plot(this_df.loc[:x_sample_nb - 2, 'x' + param + '_range'] + delta, this_df.loc[1:, 'x' + param + '_score'])
                        plt.title('X' + str(x_i + 1))
                    plt.show()

            df_x.to_pickle('../data/amazon/df_x.pkl')
            peq['q_cal'].to_pickle('../data/amazon/q_cal.pkl')